In [ ]:
import h5py
import numpy as np
import os
import scipy
import verdict

In [ ]:
import galaxy_dive.analyze_data.particle_data as particle_data
import galaxy_dive.plot_data.generic_plotter as generic_plotter
import galaxy_dive.utils.astro as astro_utils
import galaxy_dive.plot_data.plotting as plotting

In [ ]:
import galaxy_dive.utils.data_operations as data_operations

# Load Galaxy Data

In [ ]:
snum = 600
sim_data_dir = '/scratch/projects/xsede/GalaxiesOnFIRE/core/m12i_res7100/output'
halo_data_dir = '/scratch/03057/zhafen/halo_files/core/m12i_res7100'

In [ ]:
d_data = particle_data.ParticleData(
    sdir = sim_data_dir,
    halo_data_dir = halo_data_dir,
    snum = snum,
    ptype = 1,
    main_halo_id = 0,
)

In [ ]:
d_plotter = generic_plotter.GenericPlotter( d_data )

# Rotate

In [ ]:
pos = data_operations.align_axes(
    d_data.get_data( 'P' ).transpose(),
    d_data.total_ang_momentum,
)

In [ ]:
vel = data_operations.align_axes(
    d_data.get_data( 'V' ).transpose(),
    d_data.total_ang_momentum,
)

### Make sure it looks okay

In [ ]:
%matplotlib inline
fig = plt.figure( figsize=(6,6), facecolor='w' )
ax = plt.gca()

_ = ax.hist2d(
    pos[:,0],
    pos[:,1],
    bins = [ np.linspace( -10, 10, 128 ), ] * 2,
    norm = matplotlib.colors.LogNorm(),
)

ax.set_aspect( 'equal' )


# Convert to polar

In [ ]:
x = pos[:,0]
y = pos[:,1]
z = pos[:,2]

In [ ]:
r = np.sqrt( x**2. + y**2. + z**2. )
s = np.sqrt( x**2. + y**2. )
phi = np.arctan2( y, x )

In [ ]:
in_halo = r < d_data.r_vir

In [ ]:
rhat = pos / r[:,np.newaxis]
phihat = np.cross( rhat, np.array([0,0,1]) )

In [ ]:
vr = np.array([ np.dot( vel, rhat[i] ) for i, vel in enumerate( vel ) ])
vz = vel[:,2]
vphi = np.array([ np.dot( vel, phihat[i] ) for i, vel in enumerate( vel ) ])

In [ ]:
# Mass

fig = plt.figure( figsize=(6,6), facecolor='w' )
ax = plt.gca()

s_edges = np.linspace( 0, d_data.r_vir, 128 )
z_edges = np.linspace( 0, d_data.r_vir, 128 )
hist, _, _, img = ax.hist2d(
    s,
    np.abs( z ),
    bins = [ s_edges, z_edges, ],
    norm = matplotlib.colors.LogNorm(),
)

ax.set_aspect( 'equal' )


In [ ]:
# Weighted

fig = plt.figure( figsize=(6,6), facecolor='w' )
ax = plt.gca()

weighted_hist, _, _, img = ax.hist2d(
    s,
    np.abs( z ),
    weights = vphi,
    bins = [ s_edges, z_edges, ],
#     norm = matplotlib.colors.Normalize( vmin=-150, vmax=150 ),
)

ax.set_aspect( 'equal' )


In [ ]:
avg_vphi = ( weighted_hist / hist ).transpose()

In [ ]:
# Average

fig = plt.figure( figsize=(10,10), facecolor='w' )
ax = plt.gca()

img = ax.pcolormesh(
    s_edges, z_edges, avg_vphi,
    vmin = -150,
    vmax = 150,
    cmap = 'bwr',
)

ax.set_aspect( 'equal' )

plotting.add_colorbar(
    fig,
    img
)


In [ ]:
import copy

In [ ]:
vmin = -100
vmax = 0

In [ ]:
avg_vphi_smooth = copy.deepcopy( avg_vphi )
avg_vphi_smooth[avg_vphi<vmin] = vmin
avg_vphi_smooth[avg_vphi>vmax] = vmax

In [ ]:
avg_vphi_smooth = scipy.ndimage.filters.gaussian_filter( avg_vphi_smooth, 3 )

In [ ]:
# Smoothed average

fig = plt.figure( figsize=(10,10), facecolor='w' )
ax = plt.gca()

img = ax.pcolormesh(
    s_edges, z_edges, avg_vphi_smooth,
    vmin = -150,
    vmax = 150,
    cmap = 'bwr',
)

ax.set_aspect( 'equal' )

plotting.add_colorbar(
    fig,
    img
)


In [ ]:
d_data.halo_data.get_profile_data(
    'Vc'
)